In [7]:
import json
import nltk
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
#Python library for interacting with JIRA via REST APIs.
from jira import JIRA
#connect to the jira server using HTTP basic auth
jira = JIRA(basic_auth=('bsse0914@iit.du.ac.bd', 'MbaYDePZRnDgAdL062r83605'), options={'server':'https://pg-req.atlassian.net'})
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["JiraPlugin"]
print(myclient.list_database_names())

['JiraPlugin', 'admin', 'config', 'local']


In [8]:
import re
import string 
def remove_punctuation(text):
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc.lower()

def tokenize(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    porter_stemmer= PorterStemmer()
    lemmatizer=WordNetLemmatizer()

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    terms_list = []
    #stemmed_sentence = []
    
    #Stemming is a little more aggressive. It cuts off prefixes and/or endings of words
    #based on common ones. It can sometimes be helpful, but not always because often times 
    #the new word is so much a root that it loses its actual meaning. 
    #Lemmatizing, on the other hand, maps common words into one base. 
    #Unlike stemming though, it always still returns a proper word that can be found in the dictionary.
    for w in word_tokens:
        if w not in stop_words:
            #w=porter_stemmer.stem(w)
            w=lemmatizer.lemmatize(w)
            terms_list.append(w)

    #print(word_tokens)
    #print("terms list")
    #print(terms_list)
    
    return terms_list

In [14]:
def data_processing():
    global assignee_terms_map
    assignee_terms_map = {}
    global terms_assignee_map
    terms_assignee_map={}
    global terms_assignee_list
    terms_assignee_list=[]
    global account_id_map
    account_id_map={}

    projects = jira.projects()
    num_of_projects=len(projects)
    issueCount=0
    for i in range(0,num_of_projects):
        print(projects[i])
        #jql
        issues_in_proj = jira.search_issues('project='+str(projects[i]))
        print(len(issues_in_proj))
        issueCount+=len(issues_in_proj)
    print(issueCount)
    
    cnt=0
    for p in projects:
        pro_key='project='
        pro_key+=str(p.key)
        project_key=pro_key
        size = 100
        initial = 0
        issueCount=0
        while True:
            start= initial*size
            issues = jira.search_issues(project_key,  start,size)
            issueCount+=len(issues)
            if len(issues) == 0:
                break
            initial += 1
            for issue in issues:
                if issue.fields.assignee and issue.fields.assignee!=None:
                    cnt+=1
                    summary = issue.fields.summary
                    des = issue.fields.description
                    assignee = str(issue.fields.assignee)
                    account_id = str(issue.fields.assignee.accountId)
                    
                    creation_date = str(issue.fields.created)
            
                    assignee_date = {}
                    assignee_date['assignee']=assignee
                    assignee_date['date']=creation_date
        
                    des=remove_punctuation(des)
 
                    bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                    des = ''.join(i for i in des if not i in bad_chars)
                    tokenized_des=tokenize(des)
        
                    summary=remove_punctuation(summary)
 
                    bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                    summary = ''.join(i for i in summary if not i in bad_chars)
                    tokenized_summary=tokenize(summary)
               
                    #list of all terms existing in an issue
                    dupli_terms=tokenized_des+tokenized_summary
            
                    terms = list(dict.fromkeys(dupli_terms))
                
                    keylist=[]
                    for key,val in assignee_terms_map.items():
                        keylist.append(key)
                                
                    if assignee in keylist:
                        assignee_terms_map[assignee]=assignee_terms_map[assignee]+terms
                    else:
                        assignee_terms_map[assignee]=terms
                        account_id_map[assignee]=account_id
    
                    terms_assignee_list.append(assignee_date)
        
                    for term in terms:
                        if term in terms_assignee_map:
                            terms_assignee_map[term]=terms_assignee_map[term]+terms_assignee_list
                        else:
                            terms_assignee_map[term]=terms_assignee_list  
                                        
                terms_assignee_list=[]

    print("Final map:")
    print(assignee_terms_map)
    
    print("Final terms assignee map:")
    print(terms_assignee_map)

    print("Account ID map:")
    print(account_id_map)

    print(len(assignee_terms_map))
    global available_devlist
    available_devlist=[]

    for key,val in assignee_terms_map.items():
        available_devlist.append(key)
    print(available_devlist) 
    #del assignee_terms_map["None"]
    for key,val in  assignee_terms_map.items():
        print(key, "=>", val)
        #print(key)
        #print(len(val))   
    #now i have mapped the assignee ans terms
    #now need to map term => dev
    #print(len(terms_assignee_map))
    add=0
    print(terms_assignee_map)
    
    db_assignee_list = []
    for key in account_id_map:
        temp_dict={}
        temp_dict['name'] = key
        temp_dict['assignee_id'] = account_id_map[key]
        db_assignee_list.append(temp_dict)

    print("db_assignee_list: ")
    print(db_assignee_list)
    
    #{'name': 'Moumita Asad', 'assignee_id': '5f5764e4bea5be0068101b9a'}
    mycol = mydb["Assignee"]

    if mycol.count_documents({})>0:
        for i in range(0,len(db_assignee_list)):
            myquery = {}
            myquery["name"] = db_assignee_list[i]["name"]
            
            mydoc = mycol.find(myquery)
            if(mydoc.count()>0):
                print("Entry exists")
                
            else:
                print("Entry doesn't exist")
                mydict = {}
                mydict["name"] = myquery["name"]
                mydict["assignee_id"] = db_assignee_list[i]["assignee_id"]
                x = mycol.insert_one(mydict)

    else:
        print("No document yet in this collection.")
        #mydict = {}
        #mydict["name"] = db_assignee_list[i]["name"]
        #mydict["assignee_id"] = db_assignee_list[i]["assignee_id"]
        x = mycol.insert_many(db_assignee_list)
    
    #check if query exists, if not then create one 

    #print "Assignee" after the update:
    for x in mycol.find():
        print(x)

In [15]:
data_processing()

C1R
12
FAZ
5
FEP
4
RMS
0
SAAJ
8
29
Final map:
{'Moumita Asad': ['write', 'icse', 'paper', 'including', 'finding', 'regression', 'analysis', 'primary', 'demonstration', 'write', 'blog', 'productivity', 'employee', 'wfh', 'publish', 'dsse', 'website', 'explore', 'literature', 'find', 'paper', 'related', 'employee', 'productivity', 'work', 'home'], 'Noshin Tahsin': ['regression', 'analysis', 'data', 'write', 'paper', 'finding', 'far', 'arxiv', 'create', 'pie', 'chart', 'data', 'visualization', 'clean', 'preprocess', 'data', 'change', 'column', 'name', 'drop', 'invalid', 'encode', 'categorical', 'find', 'productivityrelated', 'factor', 'related', 'paper', 'prepare', 'survey', 'questionnare', 'getting', 'list', 'view', 'queried', 'result', 'user', 'vscode', 'extension', 'exploring', 'feature', 'better', 'strategy', 'viewing', 'implementing', 'search', 'viewer', 'error', 'need', 'solve', 'find', 'fix', 'bug', 'add', 'new', 'table', 'ui', 'deploy', 'backend', 'heroku', 'assignee', 'suggestion